In [9]:
# -*- coding: utf-8 -*-
import numpy as np
import os
from xml.etree import ElementTree
import subprocess
import sys
import pandas as pd

#[x_min,y_min,x_max,y_max]の4次元情報
BOX_INFO_NUM=4
#クラス数
NUM_CLASS=5#8


# 対象物体
INPUTDIRECTORY = 'museumPlate_annotation/data3'
CLASS = 'plateC'#'potC'
OUTPUT_DIRECTORY= CLASS


class XML_preprocessor(object):

    def __init__(self, data_path):
        self.path_prefix = data_path
        self.num_classes = NUM_CLASS#sign,11/23標識はその他標識を入れると認識率さがるので、除く
        self.data = {}#dict()
        self._preprocess_XML()

    def _preprocess_XML(self):
        filenames = os.listdir(self.path_prefix)
        for filename in filenames:
            print(filename)
            tree = ElementTree.parse(self.path_prefix + filename)
            root = tree.getroot()
            bounding_boxes = []
            one_hot_classes = []

            paths = []
            center_labels=[]
            
            final_dict={}
            

            width = float(720)#float(size_tree.find('width').text) #270
            height = float(405)#float(size_tree.find('height').text) #480
            for object_tree in root.findall('object'):
                # crop対象を決定
                if object_tree.find('name').text != CLASS: 
                    continue 
                for bounding_box in object_tree.findall('polygon'):
                    x=[]
                    y=[]
                    count=0
                    first_one = []
                    points=[]
                    # annotationされているか検証
                    pt=bounding_box.find('pt')
                    l=pt.find('l').text
                    # object lossの場合飛ばす
                    if l =="1":
                        continue
                    # 10フレーム毎にannotationデータを取得するため
                    #　　後ろが・・0.jpgのもののみ取得
                    t = str(bounding_box.find('t').text)
                    num = list(t)
                    
                   # if list(t)[-1] == "0":     
               
                    for pt in bounding_box.findall('pt'):
                        if count==0:
                            first_one.append(float(pt.find('x').text))
                            first_one.append(float(pt.find('y').text))
                        x.append(float(pt.find('x').text))
                        y.append(float(pt.find('y').text))
                        points.append([float(pt.find('x').text),float(pt.find('y').text)])
                        print(points)
                        count+=1
                    center= [abs(points[3][0]-points[0][0])/2+points[0][0], abs(points[1][1]-points[0][1])/2+points[0][1]]
                    xmin = min(x)
                    xmax = max(x)
                    ymin = min(y)
                    ymax = max(y)
                    bounding_box_ = [xmin, ymin, xmax, ymax]
                    bounding_boxes.append(bounding_box_)
                    print(np.shape(bounding_box_))

                    directory = 0
                    if len(num) == 2:
                        directory = 0
                    elif len(num) == 3:
                        directory = num[0]
                    elif len(num) == 4:
                        directory = num[0]+num[1]

                    file_path = '/Users/matsunagamasaaki/MasterResearch/'+INPUTDIRECTORY+'/data/frames_in/0/'+ str(directory)+'/'+t+'.jpg'
                    out_path = '/Users/matsunagamasaaki/MasterResearch/'+INPUTDIRECTORY+'/data/'+OUTPUT_DIRECTORY+'/'+t+'.jpg'
                    save_path = '/Users/matsunagamasaaki/MasterResearch/'+INPUTDIRECTORY+'/data/'+OUTPUT_DIRECTORY+'/'+t+'.jpg'

                    args = ['convert', str(file_path), '-crop', str(int(bounding_box_[2]-bounding_box_[0]))+'x'+str(int(bounding_box_[3]-bounding_box_[1]))+'+'+str(int(first_one[0]))+'+'+str(int(first_one[1])), str(out_path)]


                    try:

                        res = subprocess.run(args, stdout=subprocess.PIPE)

                        paths.append(save_path)
                        center_labels.append(center)


                    except:
                        print("ERROR")
                        import traceback
                        traceback.print_exc()
                else:
                    continue
            df = pd.DataFrame({'path':paths,'center':center_labels})
            df.to_csv('./cup_test_data.csv')            
            self.data = final_dict

    def _to_one_hot(self, name):
        one_hot_vector = [0] * self.num_classes
        case = 4 #case = 0 >all(20class),case = 1>car(9class),case = 2 >traffic sign(8class)

        if case == 3:
            if name == 'background':
                one_hot_vector[0] = 1
            elif name == 'potA':
                one_hot_vector[1] = 1
            elif name == 'potB':
                one_hot_vector[2] = 1        
            elif name == 'potC':
                one_hot_vector[3] = 1            
            elif name == 'potD':
                one_hot_vector[4] = 1             
            elif name == 'potE':
                one_hot_vector[5] = 1
            elif name == 'potF':
                one_hot_vector[6] = 1
            else:
                print('unknown label: %s' %name)
                
                
        if case == 4:
            if name == 'plateA':
                one_hot_vector[0] = 1
            elif name == 'plateB':
                one_hot_vector[1] = 1
            elif name == 'plateC':
                one_hot_vector[2] = 1
            elif name == 'plateD':
                one_hot_vector[3] = 1
            elif name == 'plateE':
                one_hot_vector[4] = 1
            
            else:
                print('unknown label: %s' %name)
        


import pickle
data = XML_preprocessor('/Users/matsunagamasaaki/MasterResearch/'+INPUTDIRECTORY+'/data/xml/')

#pickle.dump(data,open('./VOC_car.pkl','wb'))#car



output.xml
[[447.0, 90.0]]
[[447.0, 90.0], [447.0, 190.0]]
[[447.0, 90.0], [447.0, 190.0], [576.0, 190.0]]
[[447.0, 90.0], [447.0, 190.0], [576.0, 190.0], [576.0, 90.0]]
(4,)
[[446.0, 90.0]]
[[446.0, 90.0], [446.0, 190.0]]
[[446.0, 90.0], [446.0, 190.0], [575.0, 190.0]]
[[446.0, 90.0], [446.0, 190.0], [575.0, 190.0], [575.0, 90.0]]
(4,)
[[445.0, 90.0]]
[[445.0, 90.0], [445.0, 190.0]]
[[445.0, 90.0], [445.0, 190.0], [574.0, 190.0]]
[[445.0, 90.0], [445.0, 190.0], [574.0, 190.0], [574.0, 90.0]]
(4,)
[[444.0, 90.0]]
[[444.0, 90.0], [444.0, 190.0]]
[[444.0, 90.0], [444.0, 190.0], [573.0, 190.0]]
[[444.0, 90.0], [444.0, 190.0], [573.0, 190.0], [573.0, 90.0]]
(4,)
[[443.0, 90.0]]
[[443.0, 90.0], [443.0, 190.0]]
[[443.0, 90.0], [443.0, 190.0], [572.0, 190.0]]
[[443.0, 90.0], [443.0, 190.0], [572.0, 190.0], [572.0, 90.0]]
(4,)
[[442.0, 90.0]]
[[442.0, 90.0], [442.0, 190.0]]
[[442.0, 90.0], [442.0, 190.0], [571.0, 190.0]]
[[442.0, 90.0], [442.0, 190.0], [571.0, 190.0], [571.0, 90.0]]
(4,)
[[441

[[444.0, 89.0]]
[[444.0, 89.0], [444.0, 189.0]]
[[444.0, 89.0], [444.0, 189.0], [573.0, 189.0]]
[[444.0, 89.0], [444.0, 189.0], [573.0, 189.0], [573.0, 89.0]]
(4,)
[[445.0, 89.0]]
[[445.0, 89.0], [445.0, 189.0]]
[[445.0, 89.0], [445.0, 189.0], [574.0, 189.0]]
[[445.0, 89.0], [445.0, 189.0], [574.0, 189.0], [574.0, 89.0]]
(4,)
[[445.0, 88.0]]
[[445.0, 88.0], [445.0, 188.0]]
[[445.0, 88.0], [445.0, 188.0], [574.0, 188.0]]
[[445.0, 88.0], [445.0, 188.0], [574.0, 188.0], [574.0, 88.0]]
(4,)
[[446.0, 88.0]]
[[446.0, 88.0], [446.0, 188.0]]
[[446.0, 88.0], [446.0, 188.0], [575.0, 188.0]]
[[446.0, 88.0], [446.0, 188.0], [575.0, 188.0], [575.0, 88.0]]
(4,)
[[447.0, 88.0]]
[[447.0, 88.0], [447.0, 188.0]]
[[447.0, 88.0], [447.0, 188.0], [576.0, 188.0]]
[[447.0, 88.0], [447.0, 188.0], [576.0, 188.0], [576.0, 88.0]]
(4,)
[[448.0, 87.0]]
[[448.0, 87.0], [448.0, 187.0]]
[[448.0, 87.0], [448.0, 187.0], [577.0, 187.0]]
[[448.0, 87.0], [448.0, 187.0], [577.0, 187.0], [577.0, 87.0]]
(4,)
[[448.0, 87.0]]


[[438.0, 114.0]]
[[438.0, 114.0], [438.0, 214.0]]
[[438.0, 114.0], [438.0, 214.0], [567.0, 214.0]]
[[438.0, 114.0], [438.0, 214.0], [567.0, 214.0], [567.0, 114.0]]
(4,)
[[438.0, 115.0]]
[[438.0, 115.0], [438.0, 215.0]]
[[438.0, 115.0], [438.0, 215.0], [567.0, 215.0]]
[[438.0, 115.0], [438.0, 215.0], [567.0, 215.0], [567.0, 115.0]]
(4,)
[[438.0, 115.0]]
[[438.0, 115.0], [438.0, 215.0]]
[[438.0, 115.0], [438.0, 215.0], [567.0, 215.0]]
[[438.0, 115.0], [438.0, 215.0], [567.0, 215.0], [567.0, 115.0]]
(4,)
[[438.0, 116.0]]
[[438.0, 116.0], [438.0, 216.0]]
[[438.0, 116.0], [438.0, 216.0], [567.0, 216.0]]
[[438.0, 116.0], [438.0, 216.0], [567.0, 216.0], [567.0, 116.0]]
(4,)
[[438.0, 116.0]]
[[438.0, 116.0], [438.0, 216.0]]
[[438.0, 116.0], [438.0, 216.0], [567.0, 216.0]]
[[438.0, 116.0], [438.0, 216.0], [567.0, 216.0], [567.0, 116.0]]
(4,)
[[438.0, 117.0]]
[[438.0, 117.0], [438.0, 217.0]]
[[438.0, 117.0], [438.0, 217.0], [567.0, 217.0]]
[[438.0, 117.0], [438.0, 217.0], [567.0, 217.0], [567.0,

[[432.0, 134.0]]
[[432.0, 134.0], [432.0, 221.0]]
[[432.0, 134.0], [432.0, 221.0], [560.0, 221.0]]
[[432.0, 134.0], [432.0, 221.0], [560.0, 221.0], [560.0, 134.0]]
(4,)
[[432.0, 134.0]]
[[432.0, 134.0], [432.0, 221.0]]
[[432.0, 134.0], [432.0, 221.0], [560.0, 221.0]]
[[432.0, 134.0], [432.0, 221.0], [560.0, 221.0], [560.0, 134.0]]
(4,)
[[432.0, 134.0]]
[[432.0, 134.0], [432.0, 221.0]]
[[432.0, 134.0], [432.0, 221.0], [560.0, 221.0]]
[[432.0, 134.0], [432.0, 221.0], [560.0, 221.0], [560.0, 134.0]]
(4,)
[[430.0, 135.0]]
[[430.0, 135.0], [430.0, 221.0]]
[[430.0, 135.0], [430.0, 221.0], [558.0, 221.0]]
[[430.0, 135.0], [430.0, 221.0], [558.0, 221.0], [558.0, 135.0]]
(4,)
[[428.0, 135.0]]
[[428.0, 135.0], [428.0, 221.0]]
[[428.0, 135.0], [428.0, 221.0], [556.0, 221.0]]
[[428.0, 135.0], [428.0, 221.0], [556.0, 221.0], [556.0, 135.0]]
(4,)
[[426.0, 136.0]]
[[426.0, 136.0], [426.0, 222.0]]
[[426.0, 136.0], [426.0, 222.0], [554.0, 222.0]]
[[426.0, 136.0], [426.0, 222.0], [554.0, 222.0], [554.0,

[[411.0, 143.0]]
[[411.0, 143.0], [411.0, 228.0]]
[[411.0, 143.0], [411.0, 228.0], [541.0, 228.0]]
[[411.0, 143.0], [411.0, 228.0], [541.0, 228.0], [541.0, 143.0]]
(4,)
[[411.0, 143.0]]
[[411.0, 143.0], [411.0, 227.0]]
[[411.0, 143.0], [411.0, 227.0], [541.0, 227.0]]
[[411.0, 143.0], [411.0, 227.0], [541.0, 227.0], [541.0, 143.0]]
(4,)
[[411.0, 142.0]]
[[411.0, 142.0], [411.0, 227.0]]
[[411.0, 142.0], [411.0, 227.0], [541.0, 227.0]]
[[411.0, 142.0], [411.0, 227.0], [541.0, 227.0], [541.0, 142.0]]
(4,)
[[411.0, 142.0]]
[[411.0, 142.0], [411.0, 226.0]]
[[411.0, 142.0], [411.0, 226.0], [541.0, 226.0]]
[[411.0, 142.0], [411.0, 226.0], [541.0, 226.0], [541.0, 142.0]]
(4,)
[[411.0, 142.0]]
[[411.0, 142.0], [411.0, 226.0]]
[[411.0, 142.0], [411.0, 226.0], [541.0, 226.0]]
[[411.0, 142.0], [411.0, 226.0], [541.0, 226.0], [541.0, 142.0]]
(4,)
[[411.0, 141.0]]
[[411.0, 141.0], [411.0, 225.0]]
[[411.0, 141.0], [411.0, 225.0], [542.0, 225.0]]
[[411.0, 141.0], [411.0, 225.0], [542.0, 225.0], [542.0,

...

[[405.0, 61.0]]
[[405.0, 61.0], [405.0, 138.0]]
[[405.0, 61.0], [405.0, 138.0], [522.0, 138.0]]
[[405.0, 61.0], [405.0, 138.0], [522.0, 138.0], [522.0, 61.0]]
(4,)
[[405.0, 61.0]]
[[405.0, 61.0], [405.0, 138.0]]
[[405.0, 61.0], [405.0, 138.0], [522.0, 138.0]]
[[405.0, 61.0], [405.0, 138.0], [522.0, 138.0], [522.0, 61.0]]
(4,)
[[405.0, 61.0]]
[[405.0, 61.0], [405.0, 138.0]]
[[405.0, 61.0], [405.0, 138.0], [522.0, 138.0]]
[[405.0, 61.0], [405.0, 138.0], [522.0, 138.0], [522.0, 61.0]]
(4,)
[[406.0, 61.0]]
[[406.0, 61.0], [406.0, 138.0]]
[[406.0, 61.0], [406.0, 138.0], [523.0, 138.0]]
[[406.0, 61.0], [406.0, 138.0], [523.0, 138.0], [523.0, 61.0]]
(4,)
[[406.0, 61.0]]
[[406.0, 61.0], [406.0, 138.0]]
[[406.0, 61.0], [406.0, 138.0], [523.0, 138.0]]
[[406.0, 61.0], [406.0, 138.0], [523.0, 138.0], [523.0, 61.0]]
(4,)
[[407.0, 61.0]]
[[407.0, 61.0], [407.0, 138.0]]
[[407.0, 61.0], [407.0, 138.0], [523.0, 138.0]]
[[407.0, 61.0], [407.0, 138.0], [523.0, 138.0], [523.0, 61.0]]
(4,)
[[407.0, 61.0]]


[[396.0, 77.0]]
[[396.0, 77.0], [396.0, 153.0]]
[[396.0, 77.0], [396.0, 153.0], [510.0, 153.0]]
[[396.0, 77.0], [396.0, 153.0], [510.0, 153.0], [510.0, 77.0]]
(4,)
[[397.0, 77.0]]
[[397.0, 77.0], [397.0, 153.0]]
[[397.0, 77.0], [397.0, 153.0], [511.0, 153.0]]
[[397.0, 77.0], [397.0, 153.0], [511.0, 153.0], [511.0, 77.0]]
(4,)
[[398.0, 77.0]]
[[398.0, 77.0], [398.0, 153.0]]
[[398.0, 77.0], [398.0, 153.0], [512.0, 153.0]]
[[398.0, 77.0], [398.0, 153.0], [512.0, 153.0], [512.0, 77.0]]
(4,)
[[399.0, 77.0]]
[[399.0, 77.0], [399.0, 153.0]]
[[399.0, 77.0], [399.0, 153.0], [513.0, 153.0]]
[[399.0, 77.0], [399.0, 153.0], [513.0, 153.0], [513.0, 77.0]]
(4,)
[[400.0, 77.0]]
[[400.0, 77.0], [400.0, 153.0]]
[[400.0, 77.0], [400.0, 153.0], [514.0, 153.0]]
[[400.0, 77.0], [400.0, 153.0], [514.0, 153.0], [514.0, 77.0]]
(4,)
[[401.0, 77.0]]
[[401.0, 77.0], [401.0, 153.0]]
[[401.0, 77.0], [401.0, 153.0], [515.0, 153.0]]
[[401.0, 77.0], [401.0, 153.0], [515.0, 153.0], [515.0, 77.0]]
(4,)
[[402.0, 77.0]]


[[441.0, 62.0]]
[[441.0, 62.0], [441.0, 138.0]]
[[441.0, 62.0], [441.0, 138.0], [555.0, 138.0]]
[[441.0, 62.0], [441.0, 138.0], [555.0, 138.0], [555.0, 62.0]]
(4,)
[[442.0, 62.0]]
[[442.0, 62.0], [442.0, 138.0]]
[[442.0, 62.0], [442.0, 138.0], [556.0, 138.0]]
[[442.0, 62.0], [442.0, 138.0], [556.0, 138.0], [556.0, 62.0]]
(4,)
[[442.0, 61.0]]
[[442.0, 61.0], [442.0, 137.0]]
[[442.0, 61.0], [442.0, 137.0], [556.0, 137.0]]
[[442.0, 61.0], [442.0, 137.0], [556.0, 137.0], [556.0, 61.0]]
(4,)
[[443.0, 60.0]]
[[443.0, 60.0], [443.0, 136.0]]
[[443.0, 60.0], [443.0, 136.0], [557.0, 136.0]]
[[443.0, 60.0], [443.0, 136.0], [557.0, 136.0], [557.0, 60.0]]
(4,)
[[444.0, 60.0]]
[[444.0, 60.0], [444.0, 136.0]]
[[444.0, 60.0], [444.0, 136.0], [558.0, 136.0]]
[[444.0, 60.0], [444.0, 136.0], [558.0, 136.0], [558.0, 60.0]]
(4,)
[[445.0, 59.0]]
[[445.0, 59.0], [445.0, 135.0]]
[[445.0, 59.0], [445.0, 135.0], [559.0, 135.0]]
[[445.0, 59.0], [445.0, 135.0], [559.0, 135.0], [559.0, 59.0]]
(4,)
[[445.0, 58.0]]


[[399.0, 74.0]]
[[399.0, 74.0], [399.0, 150.0]]
[[399.0, 74.0], [399.0, 150.0], [513.0, 150.0]]
[[399.0, 74.0], [399.0, 150.0], [513.0, 150.0], [513.0, 74.0]]
(4,)
[[399.0, 75.0]]
[[399.0, 75.0], [399.0, 151.0]]
[[399.0, 75.0], [399.0, 151.0], [513.0, 151.0]]
[[399.0, 75.0], [399.0, 151.0], [513.0, 151.0], [513.0, 75.0]]
(4,)
[[399.0, 75.0]]
[[399.0, 75.0], [399.0, 151.0]]
[[399.0, 75.0], [399.0, 151.0], [513.0, 151.0]]
[[399.0, 75.0], [399.0, 151.0], [513.0, 151.0], [513.0, 75.0]]
(4,)
[[399.0, 75.0]]
[[399.0, 75.0], [399.0, 151.0]]
[[399.0, 75.0], [399.0, 151.0], [513.0, 151.0]]
[[399.0, 75.0], [399.0, 151.0], [513.0, 151.0], [513.0, 75.0]]
(4,)
[[398.0, 75.0]]
[[398.0, 75.0], [398.0, 151.0]]
[[398.0, 75.0], [398.0, 151.0], [512.0, 151.0]]
[[398.0, 75.0], [398.0, 151.0], [512.0, 151.0], [512.0, 75.0]]
(4,)
[[398.0, 76.0]]
[[398.0, 76.0], [398.0, 152.0]]
[[398.0, 76.0], [398.0, 152.0], [512.0, 152.0]]
[[398.0, 76.0], [398.0, 152.0], [512.0, 152.0], [512.0, 76.0]]
(4,)
[[398.0, 76.0]]


[[438.0, 93.0]]
[[438.0, 93.0], [438.0, 184.0]]
[[438.0, 93.0], [438.0, 184.0], [569.0, 184.0]]
[[438.0, 93.0], [438.0, 184.0], [569.0, 184.0], [569.0, 93.0]]
(4,)
[[443.0, 93.0]]
[[443.0, 93.0], [443.0, 185.0]]
[[443.0, 93.0], [443.0, 185.0], [574.0, 185.0]]
[[443.0, 93.0], [443.0, 185.0], [574.0, 185.0], [574.0, 93.0]]
(4,)


In [11]:
! pwd

/Users/matsunagamasaaki/MasterResearch/annotation/data
